In [10]:
import pandas as pd

In [11]:
df = pd.read_csv(r'C:\Users\KCC\Desktop\Flight_Price_Dataset_Q2.csv')
df

,departure_time,stops,arrival_time,class,duration,days_left,price
0,Evening,zero,Night,Economy,2.17,1,5953
1,Early_Morning,zero,Morning,Economy,2.33,1,5953
2,Early_Morning,zero,Early_Morning,Economy,2.17,1,5956
3,Morning,zero,Afternoon,Economy,2.25,1,5955
4,Morning,zero,Morning,Economy,2.33,1,5955
...,...,...,...,...,...,...,...
270133,Early_Morning,one,Night,Business,17.25,49,68739
270134,Morning,one,Evening,Business,10.08,49,69265
270135,Afternoon,one,Night,Business,10.42,49,77105
270136,Early_Morning,one,Evening,Business,10.00,49,81585


In [15]:
departure_time_map = {
    'Early_Morning': 1,
    'Morning': 2,
    'Afternoon': 3,
    'Evening': 4,
    'Night': 5
}
df['departure_time'] = df['departure_time'].map(departure_time_map)
df['arrival_time'] = df['arrival_time'].map(departure_time_map)


stops_map = {
    'zero': 0,
    'one': 1,
    'two_or_more':2
}
df['stops'] = df['stops'].map(stops_map)

class_map = {
    'Economy': 1,
    'Business': 2
}
df['class'] = df['class'].map(class_map)